# S2W6D1: 语言的原子化 (Tokenizers Deep Dive)

在 Transformer 看到任何数字之前，必须有一个“翻译官”将人类的自然语言（文本）转换为机器能理解的数字（Token IDs）。这个组件就是**Tokenizer**。

## 1 核心理论：从文本到 ID

### 1.1 为什么不能直接用 ASCII 码？

如果在 C 语言里，'A' 是 65。但在 AI 里，我们不能直接把句子变成 ASCII 码流。

  * **原因**: 语义稀疏。'a', 'p', 'p', 'l', 'e' 拆开看没有任何意义，只有组合在一起才有语义。
  * **解决**: 我们需要一个**词表 (Vocabulary)**，把有意义的单元（字、词、子词）映射为一个唯一的 ID。

### 1.2 分词算法之争 (Interview Point)

面试官经常问：“BERT 和 GPT 的分词有什么区别？”

  * **WordPiece (BERT 使用)**:

      * **策略**: 概率驱动。它会把不认识的长词拆解为常见的“词根”。
      * **例子**: "unhappiness" -\> "un" + "\#\#happi" + "\#\#ness" (注意 `##` 表示这是一个词的后缀，不能单独成词)。
      * **中文特例**: `bert-base-chinese` 基本上是 **字 (Character)** 级别的分词。比如 "机器人" -\> "机", "器", "人"。

  * **BPE (Byte-Pair Encoding, GPT/RoBERTa 使用)**:

      * **策略**: 统计频次驱动。不断合并出现频率最高的字符对。
      * **目的**: 都是为了解决 **OOV (Out Of Vocabulary)** 问题。如果词表里没有 "iPhone16"，就把它拆成 "iPhone" + "16"，保证模型能读懂。

### 1.3 特殊 Token (The Special Ones)

BERT 的输入不仅仅是字，还必须包含“交通信号灯”：

  * `[CLS]` (ID: 101): **Classification Token**。放在句首。
      * **面试坑**: 为什么分类任务只用这个 Token 的输出？(因为在 Pre-training 时，模型被训练为将整个句子的语义聚合到这个位置)。
  * `[SEP]` (ID: 102): **Separator Token**。用于句尾，或者区分两个句子（QA任务中的问题与文章）。
  * `[PAD]` (ID: 0): **Padding Token**。用于占位，把短句补齐长度。
  * `[UNK]` (ID: 100): **Unknown Token**。实在不认识的生僻字，就变成这个（信息的损失）。

## 2\. 代码实战 (Hands-on Code)

### 2.1 加载 Tokenizer

我们使用最经典的中文 BERT 模型。

In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com' # 临时开启镜像
from transformers import BertTokenizer

# --- 配置路径 ---
# 这一步和之前一样，定位到 data/pretrained_models/bert-base-chinese
current_dir = os.getcwd()
# 这里的 target_path 就是我们以后存放“干净”模型的地方
target_path = os.path.abspath(os.path.join(current_dir, '../../data/pretrained_models/bert-base-chinese'))

# --- 核心逻辑变化 ---
# 1. 检查目录下是否有 vocab.txt。如果有，说明已经下载并保存好了，直接加载本地。
# 2. 如果没有，说明是第一次，需要联网下载，然后“另存为”到这个目录。

if not os.path.exists(os.path.join(target_path, 'vocab.txt')):
    print("🌐 本地未发现模型，正在从镜像源下载...")
    
    # 先下载到内存/临时缓存
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    
    # 【关键一步】将干净的文件保存到指定目录
    # 这会在 target_path 下生成 vocab.txt, tokenizer_config.json 等标准文件
    tokenizer.save_pretrained(target_path)
    print(f"✅ 下载并转存完成！模型已保存在: {target_path}")
else:
    print(f"📂 发现本地模型，直接加载: {target_path}")

# --- 以后所有的代码都用下面这一行加载 ---
# 注意：这里传进去的不再是 'bert-base-chinese' 这个字符串
# 而是 target_path (绝对路径)
tokenizer = BertTokenizer.from_pretrained(target_path)

print("验证加载:", tokenizer.vocab_size)

/home/goodminton/anaconda3/envs/utils/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌐 本地未发现模型，正在从镜像源下载...
✅ 下载并转存完成！模型已保存在: /home/goodminton/study/AI-Interview-Sprint/data/pretrained_models/bert-base-chinese
验证加载: 21128


### 2.2 核心操作：Encode

我们要看看一句话是如何变成数字的。

In [3]:
text = "把红色的方块递给我。.."

# 方式 A: 简单 encode (只返回 IDs)
# add_special_tokens=True 会自动加上 [CLS] 和 [SEP]
ids = tokenizer.encode(text, add_special_tokens=True)
print("IDs:", ids)

# 方式 B: 还原回文本 (Decode)
decoded_text = tokenizer.decode(ids)
print("Decoded:", decoded_text)

# 观察：
# 101 是 [CLS]
# 102 是 [SEP]
# 511 (红) 等等...

IDs: [101, 2828, 5273, 5682, 4638, 3175, 1779, 6853, 5314, 2769, 511, 119, 119, 102]
Decoded: [CLS] 把 红 色 的 方 块 递 给 我 。.. [SEP]


### 2.3 核心操作：Tokenizer Call (工业界写法)

在实际工程中，我们会直接调用 `tokenizer()` 对象，因为它会一次性返回所有需要的 Tensor。

```python
# 模拟一个 Batch 的输入 (具身场景指令)
instructions = [
    "向左旋转30度。",
    "抓取检测到的目标。"
]

# padding=True: 按照batch中最长的句子补齐
# return_tensors='pt': 直接返回 PyTorch Tensor
inputs = tokenizer(
    instructions, 
    padding=True, 
    truncation=True, 
    max_length=20, 
    return_tensors='pt'
)

print("\nKeys inside inputs:", inputs.keys())
# 输出应该有: input_ids, token_type_ids, attention_mask

print("\n--- Input IDs (这是给 Embedding 层的输入) ---")
print(inputs['input_ids'])
# 观察 0 的存在，那就是 [PAD]

print("\n--- Attention Mask (这是给 Self-Attention 的 Mask) ---")
print(inputs['attention_mask'])
# 观察: 1 代表真实内容，0 代表 padding。
# 记住 Week 5 我们手写的 mask 吗？这就是来源！
```

-----

## 3\. 深度解析与面试坑 (Deep Dive)

### 3.1 词表大小为什么是 21128？

  * **面试题**: `bert-base-chinese` 的 21128 包含了所有汉字吗？
  * **答案**: 没有。它包含了最常用的汉字、英文字母、数字和标点。如果你输入非常生僻的古文，会被转为 `[UNK]`。
  * **对比**: 英文 BERT 的 vocab size 是 30522。

### 3.2 Attention Mask 的作用

请仔细看代码输出的 `attention_mask`。

  * 在 Week 5 手写 Attention 时，我们有一个 `mask` 参数，用来把某些位置填成 `-inf`。
  * 这里输出的 `0`，在传入模型内部后，会自动变成 `-inf`，从而让 Softmax 后的概率为 0。
  * **一句话解释**: 告诉 Attention 机制，“不要把注意力浪费在占位符 `[PAD]` 上”。

-----

## 4\. 🤖 具身智能特供：领域扩充 (Adding Tokens)

作为机器人项目负责人，你肯定会遇到标准词表里没有的专业术语。
例如：你的机器人有一个特定的动作叫 **"Servo\_Reset\_Z"**。

如果不做处理，BERT 会把它拆成：`Servo`, `_`, `Re`, `##set`, `_`, `Z`（甚至更碎），这会破坏语义的完整性。

**解决方案：`add_tokens`**

```python
# 假设这是我们的特殊指令
robot_cmd = "执行指令：Servo_Reset_Z 动作。"

print("原始分词结果:", tokenizer.tokenize(robot_cmd))
# 可能会被切得很碎，或者把 Servo 这种英文切成字母

# --- 核心操作：添加新词 ---
new_tokens = ["Servo_Reset_Z", "Joint_01"]
num_added_toks = tokenizer.add_tokens(new_tokens)

print(f"添加了 {num_added_toks} 个新 Token")

# 再次分词
print("新分词结果:", tokenizer.tokenize(robot_cmd))
# 现在 'Servo_Reset_Z' 应该作为一个整体存在了！

ids = tokenizer.encode(robot_cmd)
print("新 IDs:", ids)
# 注意：新 Token 的 ID 会排在 21128 之后
```

⚠️ **高危警告 (Critical Warning)**:
你现在只是修改了 Tokenizer（字典）。模型（大脑）的 Embedding 矩阵大小还是旧的（21128行）。
如果你直接把这个新 ID 传给模型，**模型会报错**（索引越界）。
**解决方法**：在加载模型后，必须调用 `model.resize_token_embeddings(len(tokenizer))`。这我们在 Day 3 会详细讲。

-----

## 5\. 🚀 今日总结与任务

### 📝 总结

1.  **Tokenizer** 是文本和数学的桥梁。
2.  **Input IDs** 是字的索引，**Attention Mask** 决定了模型看哪里。
3.  **Special Tokens** (`[CLS]`, `[SEP]`) 是 BERT 架构的锚点。
4.  **Add Tokens** 是垂直领域（医疗、法律、机器人）微调的必经之路。

### ✅ 任务 (Assignment)

1.  在 Notebook 中运行上述代码。
2.  **做个实验**: 输入一个很长的句子（超过 20 字），观察 `truncation=True` 和 `max_length=20` 的效果，被截断的是前面还是后面？
3.  **思考**: `token_type_ids` 全是 0，这是干嘛的？（提示：BERT 预训练有一个 Next Sentence Prediction 任务，如果是两个句子输入，第一句是0，第二句是1）。

完成实验后，请告诉我 **“Tokenizer 实验完毕”**，我们将在 Day 2 研究如何处理变长数据的 Batching 策略！